In [1]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
print(parentdir)


/home/valdecar/Documents/projects/project/parser


In [9]:
from equation import Equation
from tokenizer.lex import lex

from equation import map_tree, map_tree_postproc


Для преобразования термов wolfram в cpp можно было использовать обычный лексический анализ с помощью re: 

In [13]:

eq = Equation("U-c*D[U,{x,2}]")
eq.parse()
eq.set_default()

# make tokenization:
lexem = lex(eq.sent)

# convert lexem for cpp replacer:
eq.prefix = lexem
eq.convert_to_node()
lexem = eq.prefix

# replace:
cpp_map = [map_tree(term, eq.tree_cpp_replacer)
           for term in lexem]
out = [map_tree_postproc(term, eq.tree_cpp_replacer)
       for term in cpp_map]

print([o.cpp.out for o in out])

['source[0][idx + 0]', '-', 'params[2]', '*', '(DXM2 * (source[0][idx + 1 * Block0StrideX * Block0CELLSIZE + 0] - 2.0 * source[0][idx + 0 * Block0StrideX * Block0CELLSIZE + 0] + source[0][idx-1 * Block0StrideX * Block0CELLSIZE + 0]))']


Однако, некоторые термы не потдаются лексическому разбору, например скобочные (функции, pow). Так, в следующем примере неопределенность к какому терму принадлежит скобка "(" перед U. Поэтому возникает необходимость использовать синтаксический анализ (восходящий разбор с использованием алгоритма Кока — Янгера — Касами https://en.wikipedia.org/wiki/CYK_algorithm) 

In [25]:
eq_pow = Equation("U'=(V(t-3.1)+(U+V))^3")
eq_pow.parse()
eq_pow.set_default()
eq_pow.map_cpp()
eq_pow.show_cpp()

source[0][idx + 0]=pow(source[1][idx + 1]+(source[0][idx + 0]+source[0][idx + 1]),3)
